<a href="https://colab.research.google.com/github/RavenPillmann/ML/blob/master/Introduction_to_GANS_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from io import BytesIO
import cv2
# from PIL import Image
import PIL
from IPython.display import clear_output, Image, display


In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255., x_test / 255.

In [0]:
disc_dropout = 0.5

def discriminator():
  # Using strides rather than max-pooling. Supposedly, this works better
  model = tf.keras.models.Sequential()
  
  model.add(tf.keras.layers.Conv2D(8, kernel_size=3, input_shape=(28, 28, 1), padding='same'))
#   model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Conv2D(16, kernel_size=3, strides=2, padding='same'))
#   model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Conv2D(32, kernel_size=3, strides=2, padding='same'))
#   model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same'))
#   model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Flatten())
  
  model.add(tf.keras.layers.Dense(100))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Dense(1))
  model.add(tf.keras.layers.Activation('sigmoid'))
  
  return model
  

In [4]:
disc_model = discriminator()
disc_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
activation (Activation)      (None, 28, 28, 8)         0         
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)  

In [0]:
gen_dropout = 0.5

def generator():
  # Noise vector to 28 x 28 x 1
    
  model = tf.keras.models.Sequential()
  
  model.add(tf.keras.layers.Dense(7*7*256, input_dim=100))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  
  model.add(tf.keras.layers.Reshape((7, 7, 256)))
  model.add(tf.keras.layers.Dropout(gen_dropout))

  model.add(tf.keras.layers.UpSampling2D())
  model.add(tf.keras.layers.Conv2DTranspose(128, 5, padding='same'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  
  model.add(tf.keras.layers.UpSampling2D())
  model.add(tf.keras.layers.Conv2DTranspose(64, 5, padding='same'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  
  model.add(tf.keras.layers.Conv2DTranspose(32, 5, padding='same'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  
  model.add(tf.keras.layers.Conv2DTranspose(1, 5, padding='same'))
  model.add(tf.keras.layers.Activation('sigmoid'))
  
  
#   model.add(tf.keras.layers.Dense(100, input_shape=(100, 1)))
#   model.add(tf.keras.layers.BatchNormalization())
#   model.add(tf.keras.layers.Activation('relu'))
  
#   model.add(tf.keras.layers.Dense(1000))
#   model.add(tf.keras.layers.BatchNormalization())
#   model.add(tf.keras.layers.Activation('relu'))
  
#   model.add(tf.keras.layers.Dense(28 * 28 * 1))
#   model.add(tf.keras.layers.BatchNormalization())
#   model.add(tf.keras.layers.Activation('relu'))
  
#   model.add(tf.keras.layers.Reshape((28, 28, 1), input_shape=(28 * 28 * 1, 1)))
#   model.add(tf.keras.layers.Activation('sigmoid')) # Makes it so that output is between 0 and 1
  
  return model

In [6]:
# noise = np.random.uniform(-1, 1, 100)

gen_model = generator()
gen_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_v1 (Batc (None, 12544)             50176     
_________________________________________________________________
activation_6 (Activation)    (None, 12544)             0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       819328    
__________

In [0]:
discriminator_optimizer = tf.keras.optimizers.RMSprop(lr=4e-4)
# discriminator_optimizer = tf.keras.optimizers.Adagrad(lr=8e-5)

discriminator_model = tf.keras.models.Sequential()
discriminator_model.add(disc_model)
discriminator_model.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer, metrics=['accuracy'])

In [0]:
adversarial_optimizer = tf.keras.optimizers.RMSprop(lr=1e-4)
# adversarial_optimizer = tf.keras.optimizers.Adagrad(lr=4e-5)

adversarial_model = tf.keras.models.Sequential()
adversarial_model.add(gen_model)
adversarial_model.add(disc_model)
adversarial_model.compile(loss='binary_crossentropy', optimizer=adversarial_optimizer, metrics=['accuracy'])

In [0]:
# /device:GPU:0
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# tf.keras.backend.set_session(sess)

In [10]:
# Training the discriminator and generator
number_epochs = 20000
batch_size = 64

disc_loss = np.array([0., 0.])
adv_loss = np.array([0., 0.])

for i in range(number_epochs):
  real_images = x_train[np.random.randint(0, x_train.shape[0], size=batch_size), :, :]

  noise = np.random.uniform(-1., 1., size=[batch_size, 100])
  fake_images = gen_model.predict(noise) # Not sure if I should be using gen_model or not
  fake_images = fake_images[:, :, :, 0] # Tbh not too sure I should be reshaping like this, but it seems necessary to concatenate
    
  x = np.concatenate((real_images, fake_images)).reshape(-1, 28, 28, 1)
  y = np.ones([2 * batch_size, 1])

  y[batch_size:, :] = 0

  discriminator_model.trainable=True
  disc_loss += np.array(discriminator_model.train_on_batch(x, y))

  adv_y = np.ones([batch_size, 1])

  noise = np.random.uniform(-1., 1., size=[batch_size, 100])
  discriminator_model.trainable=False
  adv_loss += np.array(adversarial_model.train_on_batch(noise, adv_y))
  
  if i % 100 == 0:
    print("disc_loss:", (disc_loss / (i+1)), "adv_loss:", (adv_loss / (i+1)))

Instructions for updating:
Use tf.cast instead.
disc_loss: [0.69908321 0.4609375 ] adv_loss: [0.87032056 0.09375   ]
disc_loss: [0.11083174 0.96434097] adv_loss: [0.16859366 0.86741955]
disc_loss: [0.05587218 0.98208178] adv_loss: [0.08471733 0.93337998]
disc_loss: [0.0379618  0.98795681] adv_loss: [0.05657207 0.95551287]
disc_loss: [0.02870441 0.99090165] adv_loss: [0.04272464 0.96656796]
disc_loss: [0.05276435 0.98779004] adv_loss: [0.04273977 0.97205589]
disc_loss: [0.14965755 0.94796433] adv_loss: [0.04562653 0.97389767]
disc_loss: [0.19769683 0.92199759] adv_loss: [0.09178723 0.95535396]
disc_loss: [0.22182422 0.9103757 ] adv_loss: [0.22516409 0.88617743]
disc_loss: [0.26477803 0.88188471] adv_loss: [0.27349356 0.85188332]
disc_loss: [0.3065453  0.84947865] adv_loss: [0.31887757 0.81125125]
disc_loss: [0.34191342 0.81824052] adv_loss: [0.35221799 0.7837903 ]
disc_loss: [0.37144184 0.79060418] adv_loss: [0.37943957 0.76057712]
disc_loss: [0.39615088 0.76944418] adv_loss: [0.4020950

In [0]:
def showarray(a, fmt='jpeg'):
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  display(Image(data=f.getvalue()))

In [26]:
random_input = np.random.uniform(-1., 1., size=[1, 100])
generated_image = gen_model.predict(random_input)[0, :, :, 0]
generated_image = generated_image * 255.

# generated_image = cv2.resize(generated_image, (0, 0), fx=2., fy=2.) # Back to full scale

%matplotlib inline
# im = Image.fromarray(generated_image)
# im.show()
showarray(generated_image)

In [0]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())